<a href="https://colab.research.google.com/github/mtxslv/digitalprocessingsimulations/blob/master/IIRstopbandFilter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from scipy import signal
import numpy as np

## Especificações do filtro IIR
Pro filtro IIR, é necessário sabermos a variação que nosso sinal vai sofrer (pulsação) na banda de passagem e na banda de rejeição. Chamemos duas variáveis, epsilon (pra banda de passagem) e delta (pra banda de rejeição). De forma a compararmos com o [filtro FIR anteriormente realizado](https://github.com/mtxslv/digitalprocessingsimulations/blob/master/band_stopFilter.ipynb), faremos:

In [0]:
# Limites de pulsação
epsilon = 0.02 # Na banda de passagem
delta = 0.01 # Na banda de rejeição

##Cálculo dos amortecimentos (dB) e das frequências importantes
Ora, os amortecimentos são importantes pra se calcular a ordem do [filtro](https://en.wikipedia.org/wiki/Butterworth_filter). As frequências de rejeição e passagem nos indicam onde "cortar" as componentes do sinal.

In [0]:
ganho_pas = 20*np.log10(epsilon) # ~-34
ganho_rej = 20*np.log10(delta) # -40

In [0]:
omega_pas = [0.35, 0.65]
omega_rej = [0.4, 0.6]

## Cálculo da ordem do filtro
Para o cálculo da ordem do filtro, usaremos a função **signal.buttord**. Para mais informações sobre, [acesse aqui](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.signal.buttord.html). 

In [0]:
N, Wn = signal.buttord(omega_pas, omega_rej, ganho_pas, ganho_rej) # N = 2, Wn = [0.08618914 0.91381099]

##Agora vamos ao *design* do filtro

Como estamos lidando com filtro butterworth, vamos utilizar uma [função pronta](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.signal.butter.html) para o cálculo dos coeficientes do polinômio.

In [0]:
b,a = signal.butter(N,Wn,'bandstop') 
# b = [5.24319516e-02 1.61394248e-07 1.04863903e-01 1.61394248e-07 5.24319516e-02]
# a = [1.00000000e+00  5.72517329e-07 -1.25602799e+00 -2.49728833e-07 4.65755795e-01]

##Vamos as equações de diferenças!!
O resultado acima alcançado já nos dá os coeficientes $b_k$ e $a_k$ da equação:

 >>>  $H(z) = \frac{Y(z)}{X(z)} = \frac{B(z)}{A(z)}=   \frac{\sum_{k=0}^{M}b_kz^-k}{1 - \sum_{k=1}^{N}a_kz^-k}  $
 
 A partir da transformada $Z$ inversa, chegamos a:
 >>> $ y[n] = \sum_{k=1}^Na_ky[n-k] + \sum_{k=0}^{M}b_kx[n-k]$

In [0]:
#TODO